# Step1) Import the raw data and preprocess them

## I) Regular exogenous data

First we recreate the higher timeframes form existing lower ones and save them in the $\color{red}{\text{RepairedExoData}}$ folder

Second we calculate all the available indicators using these data with proper colprefix and then we interpolate the missing candles with ZeroOrderHold and save the result in the $\color{red}{\text{PreprocessedData}}$ folder

## II) Endogenous data

### Preprocessing the features

First we interpolate the 1m data to use it for up_first detection of higher time frame data

Second we call the BtcPreprocessor class on which ever timeframe data we want to use, this class interpolates the missing candles, addes the up_first and calculated all the indicators. then it saves two versions of the endogenous_features in the $\color{red}{\text{PreprocessedData}}$ folder (one without any dropping to calculate the targets from it later, and one which drops the first 70 and last 1 candles to be used with target1).

### Target definitions

Third we call the TargetExtractor which calculates the target1 based on kf, from endogenous_features without any drops.
then we save the targets in the $\color{red}{\text{PreprocessedData}}$ folder.

## III) Mixing

The DataMixer class takes both the target, timeframe and exogenous args and $\color{red}{\underline{\color{white}{\text{df_name}}}}$ arg, the first three are used to choose the proper dfs from PreprocessedData folder and the last one is used to save the generated df.

In this step we just call the mixer function with proper first_candle and last_candle ($\color{lightblue}{\text{given in the Available data notebook}}$) to crop the common part of the data and desired Exogenous arg to mix and save just the endogenous or mix of endogenous and the corresponding exogenous data into $\color{red}{\underline{\color{white}{\text{df_name}}}}$_features and $\color{red}{\underline{\color{white}{\text{df_name}}}}$_labels file in the $\color{red}{\text{MixedData}}$ folder.

## IV) Feature extraction

### FeatureEngineer

FeatureEngineer class takes both df_name and model name arguments because we might have multiple models with different names that use the same features and targets, which is specified through the df_name arg.

this class loads the dfs through df_name and writes the most appropariate features for the given target in the $\color{red}{\text{models_feature_names_files}}$ with the given model_name arg

### FeatureExtractor

FeatureExtractor class first takes the $\color{red}{\underline{\color{white}{\text{df_name}}}}$ arg to initiate, in this step the features and the targets are loaded in the generated instance.

Then when you want to get the features and the labels you must give the "extract_features" function the intended $\color{red}{\underline{\color{white}{\text{model_name}}}}$, so that the feature extractor read the models features_names and separate the selected features.

if you choose to set save arg of the function to True, the function would also save the extracted features and corresponding labels in the $\color{red}{\text{ProcessedData}}$ folder with the given $\color{red}{\underline{\color{white}{\text{model_name}}}}$.

### TrainTestValidationLoader

TrainTestValidationLoader, takes the ready_features or the corresponding model_name and prepares the data for training the model you want to train. you can either use its functions to load the data for LSTM networks or use the classes variables directly if you want to reshape them in another way.
If you want to use data from different timeframes in your prediction the original_time_frame parameter should be specified, the timeframe of the provided data would be implied from the first two rows of the features_df.

# Step2) Process the data for the desired model

First we call the FeatureEngineer with proper argumans of use_exogenous_data and model_name to remove the features with too much correlation and save the remaining features names in a file with the given model name.

Forth FeatureEngineer operates on BTCUSDT_xm_data_with_up_first.It first calcualtes all the stationary indicators and then does a feature selection based on mutual information between features and target variable and correlation between features.
The last row of features is droped because for the last row, we dont have any label.Also The first 70 rows of features and targets are droped too, since during the calculation of indicators some of indicators need the previous data to be generated.
If you want to use data from different timeframes in your prediction the timeframe and original timeframe in minutes parameters are going to be different and also more data is going to be removed the beggining and the end of features and labels so that all timeframes contain the same data.
The output of this process is the BTCUSDT_xm_all_features at PreprocessedData and BTCUSDT_xm_smoothed_changes at ProcessedData plus the features_of_model_name.txt at models_feature_names_files folder.

Fourth FeatureExtractor takes the BTCUSDT_xm_all_features and features_of_model_name.txt file and extracts the chosen features, if the save parameter is set to true, the extracted data would be saved as BTCUSDT_xm_ready_features at ProcessedData folder.

Fifth TrainTestValidationLoader, takes the ready_features or its address and the targets address and prepares the data for training the model you want to train. you can ither use its functions to load the data for LSTM networks or use the classes variables directly if you want to reshape them in another way.
If you want to use data from different timeframes in your prediction the original_time_frame parameter should be specified, the timeframe of the provided data would be implied from the first two rows of the df.

# Step2) Import the predicted values by the networks

## Data that is already separated into train, test and validation

If you want to try a rule based strategy you could use "get_predictions_dfs" to get train, test, and validation data separately.

## All of the data in one df

If you use "get_predictions_df" all of the data would be provided in one single df, this option is more preferable if you want to train a ML model to predict the market based on the trend predictions of the previous model. 